In [1]:
import numpy as np
import pandas as pd
from pandas.compat import StringIO

In [2]:
pwd

'/Users/viveksasikumar/Downloads/Python-Data-Science-and-Machine-Learning-Bootcamp/Machine Learning Sections/Recommender-Systems'

In [3]:
d = open('/Users/viveksasikumar/Downloads/jester_dataset_2/jester_ratings.dat','r')
d

<_io.TextIOWrapper name='/Users/viveksasikumar/Downloads/jester_dataset_2/jester_ratings.dat' mode='r' encoding='UTF-8'>

In [4]:
df = pd.read_csv(d,sep='\s+', names= ['User Id', 'Item Id', 'Rating'])

In [5]:
df.index =df['Item Id']

In [6]:
df.head()

,User Id,Item Id,Rating
Item Id,,,
5,1,5,0.219
7,1,7,-9.281
8,1,8,-9.281
13,1,13,-6.781
15,1,15,0.875


In [7]:
f = open('/Users/viveksasikumar/Downloads/jester_dataset_2/jester_items.dat','r')
f

<_io.TextIOWrapper name='/Users/viveksasikumar/Downloads/jester_dataset_2/jester_items.dat' mode='r' encoding='UTF-8'>

In [8]:
import re

with open('/Users/viveksasikumar/Downloads/jester_dataset_2/jester_items.dat','rU') as joke_data:
    joke_set = dict()
    joke_list = joke_data.read().split('\n\n')
    for joke in joke_list[0:150]:
        joke = joke.replace('\n','')
        result = re.search(r'(\d*?):(.*)', joke)
        joke_set[int(result.group(1))] = result.group(2)
        


/Users/viveksasikumar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 'U' mode is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
joke_set.keys()
temp = []
for key, value in joke_set.items():
    aValue = value
    temp.append(aValue)


In [10]:
df2 = pd.DataFrame(data = temp,columns=['Joke'], index=joke_set.keys())

In [11]:
df2.index.name = 'Item Id'
df2['Item Id'] = df2.index
df2.head()

,Joke,Item Id
Item Id,,
1,"<p>A man visits the doctor. The doctor says, &...",1
2,<p>This couple had an excellent relationship g...,2
3,<p>Q. What&#039;s 200 feet long and has 4 teet...,3
4,<p>Q. What&#039;s the difference between a man...,4
5,<p>Q. What&#039;s O. J. Simpson&#039;s web add...,5


In [12]:
df = pd.merge(df, df2, on='Item Id')

In [13]:
df.head()

,User Id,Item Id,Rating,Joke
0,1,5,0.219,<p>Q. What&#039;s O. J. Simpson&#039;s web add...
1,2,5,-9.688,<p>Q. What&#039;s O. J. Simpson&#039;s web add...
2,3,5,-9.844,<p>Q. What&#039;s O. J. Simpson&#039;s web add...
3,4,5,-5.812,<p>Q. What&#039;s O. J. Simpson&#039;s web add...
4,5,5,6.906,<p>Q. What&#039;s O. J. Simpson&#039;s web add...


# Simple Recommender

In [14]:
C = df['Rating'].mean()
t = df.groupby('Item Id')['User Id'].count()
s = df.groupby('Item Id')['Rating'].mean()

t1 = pd.DataFrame(t)
s1 = pd.DataFrame(s)
t1.head()
s1.head()

,Rating
Item Id,
5,-1.756331
7,-1.809230
8,-0.672010
13,-0.590224
15,-1.377098


In [15]:
m = float(t.mean())
print(m)

12581.707142857143


In [16]:
t1['Item Id'] = t1.index
s1['Item Id'] = s1.index
t1.columns = ['User Id Count', 'Item Id']
t1.head()
s.columns = ['Avg Ratings', 'Item Id']
s1.head()

,Rating,Item Id
Item Id,,
5,-1.756331,5
7,-1.809230,7
8,-0.672010,8
13,-0.590224,13
15,-1.377098,15


In [17]:
df = pd.merge(df, t1, on='Item Id')
df = pd.merge(df, s1, on='Item Id' )
df.head()
df.shape

(1761439, 6)

In [18]:
q_df = df.copy().loc[df['User Id Count'] >= m]

In [19]:
q_df.shape

(1088590, 6)

In [20]:
q_df.head()

,User Id,Item Id,Rating_x,Joke,User Id Count,Rating_y
661,1,7,-9.281,<p>How many feminists does it take to screw in...,59122,-1.80923
662,2,7,9.938,<p>How many feminists does it take to screw in...,59122,-1.80923
663,3,7,-9.844,<p>How many feminists does it take to screw in...,59122,-1.80923
664,4,7,-4.500,<p>How many feminists does it take to screw in...,59122,-1.80923
665,5,7,4.750,<p>How many feminists does it take to screw in...,59122,-1.80923


In [21]:
def weighted_rating(x, m=m, C=C):
    v = x['User Id Count']
    R = x['Rating_y']
    return (v/(v+m)*R) + (m/(m+v)*C)

In [22]:
q_df1 = q_df.pivot_table(values=None, index=['User Id Count', 'Rating_y', 'Item Id'])
q_df1.head()

,,,Rating_x,User Id
User Id Count,Rating_y,Item Id,,
12934,2.370762,121,2.370762,32973.041905
13077,2.995237,63,2.995237,34862.023170
14337,2.907676,97,2.907676,34479.743531
14348,2.801907,111,2.801907,33920.577223
14399,3.046279,29,3.046279,34359.448642


In [23]:
q_df2 = q_df1.reset_index()

In [24]:
q_df2.head()

,User Id Count,Rating_y,Item Id,Rating_x,User Id
0,12934,2.370762,121,2.370762,32973.041905
1,13077,2.995237,63,2.995237,34862.023170
2,14337,2.907676,97,2.907676,34479.743531
3,14348,2.801907,111,2.801907,33920.577223
4,14399,3.046279,29,3.046279,34359.448642


In [25]:
q_df2.drop(['Rating_x','User Id'], inplace=True, axis=1)

In [26]:
q_df2.head()

,User Id Count,Rating_y,Item Id
0,12934,2.370762,121
1,13077,2.995237,63
2,14337,2.907676,97
3,14348,2.801907,111
4,14399,3.046279,29


In [27]:
q_df2['score'] = q_df2.apply(weighted_rating, axis=1)

In [28]:
q_df2 = q_df2.sort_values('score', ascending=False)

In [29]:
q_df2 = pd.merge(q_df2, df2, on='Item Id')

In [30]:
q_df2.head(15)

,User Id Count,Rating_y,Item Id,score,Joke
0,23102,3.714381,53,2.975431,<p>One Sunday morning William burst into the l...
1,25996,3.606506,89,2.958172,<p>A radio conversation between a US naval shi...
2,21822,3.711223,105,2.945935,<p>A couple of hunters are out in the woods in...
3,22519,3.504739,32,2.828661,<p>A man arrives at the gates of heaven. St. P...
4,20524,3.560305,35,2.822368,<p>An explorer in the deepest Amazon suddenly ...
5,19509,3.583496,129,2.813126,<p>A group of girlfriends is on vacation when ...
6,19421,3.505712,72,2.763804,"<p>On the first day of college, the Dean addre..."
7,19793,3.422796,104,2.721637,"<p>As a pre-med student, I had to take a diffi..."
8,22253,3.337933,106,2.716940,<p>An engineer dies and reports to the pearly ...
9,25662,3.192799,114,2.674908,<p>Sherlock Holmes and Dr. Watson go on a camp...


In [31]:
q_df2.tail()

,User Id Count,Rating_y,Item Id,score,Joke
40,56333,-0.590224,13,-0.186960,<p>They asked the Japanese visitor if they hav...
41,57720,-0.672010,8,-0.262066,<p>Q. Did you hear about the dyslexic devil wo...
42,54932,-1.377098,15,-0.818826,<p>Q: What did the blind person say when given...
43,54150,-1.587044,16,-0.982646,<p>Q. What is orange and sounds like a parrot?...
44,59122,-1.809230,7,-1.207755,<p>How many feminists does it take to screw in...


# Content Based

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer #Term Frequency-Inverse Document Frequency (TF-IDF) vectors 
tfidf = TfidfVectorizer(stop_words='english')
q_df2['Joke'] = q_df2['Joke'].fillna('')
tfidf_matrix = tfidf.fit_transform(q_df2['Joke'])
tfidf_matrix.shape

(45, 1077)

In [33]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [34]:
indices = pd.Series(q_df2.index, index=q_df2['Item Id']).drop_duplicates()

In [35]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return q_df2['Item Id'].iloc[movie_indices]

In [40]:
x = input('Enter the joke id: ')
x = int(x)

Enter the joke id: 8


In [41]:
x1 = get_recommendations(x,cosine_sim)

In [42]:
x2 = pd.DataFrame(x1, columns=['Item Id'])
x3 = pd.merge(x2,df2, on='Item Id')
x3

,Item Id,Joke
0,15,<p>Q: What did the blind person say when given...
1,68,<p>A man piloting a hot air balloon discovers ...
2,117,<p>A man joins a big corporate empire as a tra...
3,50,<p>A guy goes into confession and says to the ...
4,105,<p>A couple of hunters are out in the woods in...
5,93,"<p>Reaching the end of a job interview, the hu..."
6,148,"<p>Recently a teacher, a garbage collector, an..."
7,47,<p>There was an engineer who had an exceptiona...
8,89,<p>A radio conversation between a US naval shi...
9,49,<p>Three engineering students were gathered to...
